In [1]:
import os
import re
import numpy as np
import pydot
import rmgpy.tools.fluxdiagram
from rmgpy.solver.base import TerminationTime
from rmgpy.solver.liquid import LiquidReactor
from rmgpy.kinetics.diffusionLimited import diffusion_limiter

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
%matplotlib inline

## Settings

In [2]:
# Options controlling the individual flux diagram renderings:
program = 'dot'  # The program to use to lay out the nodes and edges
max_node_count = 50  # The maximum number of nodes to show in the diagram
max_edge_count = 50  # The maximum number of edges to show in the diagram
concentration_tol = 1e-6  # The lowest fractional concentration to show (values below this will appear as zero)
species_rate_tol = 1e-6  # The lowest fractional species rate to show (values below this will appear as zero)
max_node_pen_width = 10.0  # The thickness of the border around a node at maximum concentration
max_edge_pen_width = 10.0  # The thickness of the edge at maximum species rate
radius = 1  # The graph radius to plot around a central species
central_reaction_count = None  # The maximum number of reactions to draw from each central species (None draws all)
# If radius > 1, then this is the number of reactions from every species

# Options controlling the ODE simulations:
initial_time = 1e-12  # The time at which to initiate the simulation, in seconds
time_step = 10 ** 0.1  # The multiplicative factor to use between consecutive time points
abs_tol = 1e-16  # The absolute tolerance to use in the ODE simluations
rel_tol = 1e-8  # The relative tolerance to use in the ODE simulations

# Options controlling the generated movie:
video_fps = 6  # The number of frames per second in the generated movie
initial_padding = 5  # The number of seconds to display the initial fluxes at the start of the video
final_padding = 5  # The number of seconds to display the final fluxes at the end of the video


species_path = None
java = False            # always False
settings = None
chemkin_output = ''     # this will be generated automatically
central_species_list = None
superimpose = False     # this will always be false, delete it
save_states = False
read_states = False     # fine to keep this always false and delete relevant code below
diffusion_limited = True
check_duplicates = True

## Load Mechanisms

In [3]:
rmg_input_file = '/home/moon/autoscience/fuels/butane_20240126/simplified_input.py'  # for conditions

mech_1_inp = '/home/moon/autoscience/fuels/butane_20240126/chem_annotated.inp'
mech_1_dict = '/home/moon/autoscience/fuels/butane_20240126/species_dictionary.txt'


mech_2_inp = '/home/moon/autoscience/fuels/butane_20240315/chem_annotated.inp'
mech_2_dict = '/home/moon/autoscience/fuels/butane_20240315/species_dictionary.txt'

# mech_2_inp = '/home/moon/autoscience/aramco/truncated/aramco_truncated.inp'
# mech_2_dict = '/home/moon/autoscience/aramco//species_dictionary.txt'


if species_path is None:
    species_path = os.path.join(os.path.dirname(rmg_input_file), 'species')
    generate_images = True
else:
    generate_images = False

print('Loading RMG job 1...')
rmg_job1 = rmgpy.tools.fluxdiagram.load_rmg_job(
    rmg_input_file,
    mech_1_inp,
    mech_1_dict,
    generate_images=generate_images,
    check_duplicates=check_duplicates
)

print('Loading RMG job 2...')
rmg_job2 = rmgpy.tools.fluxdiagram.load_rmg_job(
    rmg_input_file,
    mech_2_inp,
    mech_2_dict,
    generate_images=generate_images,
    check_duplicates=check_duplicates
)

Loading RMG job 1...


┌ Warning: Replacing docs for `ReactionMechanismSimulator.getpairs :: Tuple{T} where T<:ReactionMechanismSimulator.AbstractReaction` in module `ReactionMechanismSimulator`
└ @ Base.Docs docs/Docs.jl:240
┌ Warning: Replacing docs for `ReactionMechanismSimulator.getsimilarity :: Union{Tuple{T2}, Tuple{T}, Tuple{T, T2}} where {T<:ReactionMechanismSimulator.AbstractSpecies, T2<:ReactionMechanismSimulator.AbstractSpecies}` in module `ReactionMechanismSimulator`
└ @ Base.Docs docs/Docs.jl:240
┌ Warning: Replacing docs for `SciMLBase.sol :: Union{Tuple, Tuple{D}, Tuple{S}, Tuple{N}, Tuple{T}} where {T, N, S, D}` in module `SciMLBase`
└ @ Base.Docs docs/Docs.jl:240


┌ Warning: DiffEqOperators.jl is in the process of being deprecated.
│ 
│        - For automated PDE discretization, see MethodOfLines.jl
│ 
│        - For MatrixFreeOperators, and other non-derivative operators, see SciMLOperators.jl
│ 
│        - For VecJacOperators and JacVecOperators, see SparseDiffTools.jl
└ @ DiffEqOperators ~/anaconda3/envs/rmg_env/share/julia/packages/DiffEqOperators/lHq9u/src/DiffEqOperators.jl:61
┌ Warning: No working GUI backend found for matplotlib
└ @ PyPlot ~/anaconda3/envs/rmg_env/share/julia/packages/PyPlot/H01LC/src/init.jl:153


┌ Warning: Replacing docs for `ReactionMechanismSimulator.plotrops :: Union{Tuple{X}, Tuple{Y}, Tuple{Y, X}} where {Y<:ReactionMechanismSimulator.Simulation, X<:AbstractString}` in module `ReactionMechanismSimulator`
└ @ Base.Docs docs/Docs.jl:240
┌ Warning: Replacing docs for `ReactionMechanismSimulator.plotrops :: Union{Tuple{X}, Tuple{Y}, Tuple{Y, X}} where {Y<:ReactionMechanismSimulator.Simulation, X<:AbstractString}` in module `ReactionMechanismSimulator`
└ @ Base.Docs docs/Docs.jl:240


Loading RMG job 2...


## Simulation

In [4]:
print('Conducting simulation of reaction 1')
times1, concentrations1, reaction_rates1 = rmgpy.tools.fluxdiagram.simulate(
    rmg_job1.reaction_model,
    rmg_job1.reaction_systems[0],
    settings
)

print('Conducting simulation of reaction 2')
times2, concentrations2, reaction_rates2 = rmgpy.tools.fluxdiagram.simulate(
    rmg_job2.reaction_model,
    rmg_job2.reaction_systems[0],
    settings
)

Conducting simulation of reaction 1
Conducting simulation of reaction 2


In [18]:
times1.shape

(121,)

In [16]:
reaction_rates1.shape

(121, 2984)

In [6]:
# Don't assume the mechanisms are identically named. Use the species dictionary

In [59]:
# create a universal species order for the giant flux matrix. it will be:
# [species_list1 remaining_species_in_list_2]
species_list1 = rmg_job1.reaction_model.core.species[:]
species_list2 = rmg_job2.reaction_model.core.species[:]

reaction_list1 = rmg_job1.reaction_model.core.reactions[:]
reaction_list2 = rmg_job2.reaction_model.core.reactions[:]

universal_species_list = []
for sp in species_list1:
    universal_species_list.append(sp)

list2_to_universal = {}

for i2, sp2 in enumerate(species_list2):
#     print(i2 + 1, '/', len(species_list2))
    for i1, sp1 in enumerate(species_list1):
        if sp2.is_isomorphic(sp1):
            list2_to_universal[i2] = i1
            
            
            # You also need to replace all occurences of this species in the list of reactions. Yikes.
            for r in range(len(reaction_list2)):
                for p in range(len(reaction_list2[r].products)):
                    if sp2.is_isomorphic(reaction_list2[r].products[p]):
                        reaction_list2[r].products[p] = sp1
                        reaction_list2[r].generate_pairs()
                for p in range(len(reaction_list2[r].reactants)):
                    if sp2.is_isomorphic(reaction_list2[r].reactants[p]):
                        reaction_list2[r].reactants[p] = sp1
                        reaction_list2[r].generate_pairs()
            
            break
    else:
        list2_to_universal[i2] = len(universal_species_list)
        universal_species_list.append(sp2)

num_species = len(universal_species_list)
print(f'List 1 has {len(species_list1)} species')
print(f'List 2 has {len(species_list2)} species')
print(f'There are {len(universal_species_list)} unique species')


List 1 has 164 species
List 2 has 127 species
There are 173 unique species


In [49]:
# Do the same thing for reactions

universal_reaction_list = []
for rxn in reaction_list1:
    universal_reaction_list.append(rxn)

rxn2_to_universal = {}

for i2, rxn2 in enumerate(reaction_list2):
    for i1, rxn1 in enumerate(reaction_list1):        
        if rxn2.is_isomorphic(rxn1):
            rxn2_to_universal[i2] = i1
            break
    else:
        rxn2_to_universal[i2] = len(universal_reaction_list)
        universal_reaction_list.append(rxn2)

print(f'List 1 has {len(reaction_list1)} reactions')
print(f'List 2 has {len(reaction_list2)} reactions')
print(f'There are {len(universal_reaction_list)} unique reactions')


List 1 has 2984 reactions
List 2 has 1986 reactions
There are 3409 unique reactions


In [50]:
# translate concentrations and reactions rates into the universal array form
universal_rates1 = np.zeros((reaction_rates1.shape[0], len(universal_reaction_list)))
universal_rates2 = np.zeros((reaction_rates2.shape[0], len(universal_reaction_list)))

universal_rates1[:, 0:reaction_rates1.shape[1]] = reaction_rates1
for i in range(0, reaction_rates2.shape[1]):
    universal_index = rxn2_to_universal[i]
    universal_rates2[:, universal_index] = reaction_rates2[:, i]
    
# -------------------- repeat for species
universal_concentrations1 = np.zeros((concentrations1.shape[0], len(universal_species_list)))
universal_concentrations2 = np.zeros((concentrations2.shape[0], len(universal_species_list)))

universal_concentrations1[:, 0:concentrations1.shape[1]] = concentrations1
for i in range(0, concentrations2.shape[1]):
    universal_index = list2_to_universal[i]
    universal_concentrations2[:, universal_index] = concentrations2[:, i]

In [60]:
# Compute the rates between each pair of species (big matrix warning!)
species_rates1 = np.zeros((len(times1), num_species, num_species), np.float64)
for index, reaction in enumerate(universal_reaction_list):
    rate = universal_rates1[:, index]
    if not reaction.pairs: reaction.generate_pairs()
    for reactant, product in reaction.pairs:
        reactant_index = universal_species_list.index(reactant)
        product_index = universal_species_list.index(product)
        species_rates1[:, reactant_index, product_index] += rate
        species_rates1[:, product_index, reactant_index] -= rate
        
species_rates2 = np.zeros((len(times2), num_species, num_species), np.float64)
for index, reaction in enumerate(universal_reaction_list):
    rate = universal_rates2[:, index]
    if not reaction.pairs: reaction.generate_pairs()
    for reactant, product in reaction.pairs:
        reactant_index = universal_species_list.index(reactant)
        product_index = universal_species_list.index(product)
        species_rates2[:, reactant_index, product_index] += rate
        species_rates2[:, product_index, reactant_index] -= rate

In [61]:
# Determine the maximum concentration for each species and the maximum overall concentration
max_concentrations1 = np.max(np.abs(concentrations1), axis=0)
max_concentration1 = np.max(max_concentrations1)

# Determine the maximum reaction rates
max_reaction_rates1 = np.max(np.abs(universal_rates1), axis=0)

# Determine the maximum rate for each species-species pair and the maximum overall species-species rate
max_species_rates1 = np.max(np.abs(species_rates1), axis=0)
max_species_rate1 = np.max(max_species_rates1)
species_index1 = max_species_rates1.reshape((num_species * num_species)).argsort()



# Determine the maximum concentration for each species and the maximum overall concentration
max_concentrations2 = np.max(np.abs(concentrations2), axis=0)
max_concentration2 = np.max(max_concentrations2)

# Determine the maximum reaction rates
max_reaction_rates2 = np.max(np.abs(universal_rates2), axis=0)

# Determine the maximum rate for each species-species pair and the maximum overall species-species rate
max_species_rates2 = np.max(np.abs(species_rates2), axis=0)
max_species_rate2 = np.max(max_species_rates2)
species_index2 = max_species_rates2.reshape((num_species * num_species)).argsort()

In [102]:
nodes = []  # contains all
nodes1 = []
nodes2 = []

edges = []  # contains all
edges1 = []
edges2 = []

In [103]:
for i in range(num_species * num_species):
    product_index, reactant_index = divmod(species_index1[-i - 1], num_species)
    if reactant_index > product_index:
        # Both reactant -> product and product -> reactant are in this list,
        # so only keep one of them
        continue
    if max_species_rates1[reactant_index, product_index] == 0:
        break
    if reactant_index not in nodes1 and len(nodes) < max_node_count: nodes1.append(reactant_index)
    if product_index not in nodes1 and len(nodes) < max_node_count: nodes1.append(product_index)
    if [reactant_index, product_index] not in edges1 and [product_index, reactant_index] not in edges1:
        edges1.append([reactant_index, product_index])
    if len(nodes1) > max_node_count:
        break
    if len(edges1) >= max_edge_count:
        break
        
for i in range(num_species * num_species):
    product_index, reactant_index = divmod(species_index2[-i - 1], num_species)
    if reactant_index > product_index:
        # Both reactant -> product and product -> reactant are in this list,
        # so only keep one of them
        continue
    if max_species_rates2[reactant_index, product_index] == 0:
        break
    if reactant_index not in nodes2 and len(nodes2) < max_node_count: nodes2.append(reactant_index)
    if product_index not in nodes2 and len(nodes2) < max_node_count: nodes2.append(product_index)
    if [reactant_index, product_index] not in edges2 and [product_index, reactant_index] not in edges2:
        edges2.append([reactant_index, product_index])
    if len(nodes2) > max_node_count:
        break
    if len(edges2) >= max_edge_count:
        break


In [104]:
nodes = nodes1 + nodes2
edges = edges1 + edges2

In [105]:
len(nodes)

72

In [106]:
# Create the master graph
# First we're going to generate the coordinates for all of the nodes; for
# this we use the thickest pen widths for all nodes and edges
graph = pydot.Dot('flux_diagram', graph_type='digraph', overlap="false")
graph.set_rankdir('LR')
graph.set_fontname('sans')
graph.set_fontsize('10')

# Add a node for each species
for index in nodes:
#     species = species_list1[index]
    species = universal_species_list[index]
    node = pydot.Node(name=str(species))
    node.set_penwidth(max_node_pen_width)
    graph.add_node(node)
    # Try to use an image instead of the label
    species_index = str(species) + '.png'
    image_path = ''
    if not species_path or not os.path.exists(species_path):
        continue
    for root, dirs, files in os.walk(species_path):
        for f in files:
            if f.endswith(species_index):
                image_path = os.path.join(root, f)
                break
    if os.path.exists(image_path):
        node.set_image(image_path)
        node.set_label(" ")
# Add an edge for each species-species rate
for reactant_index, product_index in edges1:
    if reactant_index in nodes and product_index in nodes:
        reactant = universal_species_list[reactant_index]
        product = universal_species_list[product_index]
        my_red = '#ff000080'
        edge = pydot.Edge(str(reactant), str(product), color=my_red)
        edge.set_penwidth(max_edge_pen_width)
        graph.add_edge(edge)
        
for reactant_index, product_index in edges2:
    if reactant_index in nodes and product_index in nodes:
        reactant = universal_species_list[reactant_index]
        product = universal_species_list[product_index]
        my_blue = '#0000ff80'
        edge = pydot.Edge(str(reactant), str(product), color=my_blue)
        edge.set_penwidth(max_edge_pen_width)
        graph.add_edge(edge)

# Generate the coordinates for all of the nodes using the specified program
graph = pydot.graph_from_dot_data(graph.create_dot(prog=program).decode('utf-8'))[0]

In [107]:
# # clear all edges
# for nodeA in graph.get_nodes():
#     for nodeB in graph.get_nodes():
#         graph.del_edge(nodeA, nodeB)
#         graph.del_edge(nodeA, nodeB)

In [112]:
len(times1)

121

In [113]:
len(times2)

121

In [117]:
assert len(times1) == len(times2)
for i in range(len(times1)):
    if not np.isclose(times1[i], times2[i]):
        print(i, times1[i], times2[i])
#     assert np.isclose(times1[i], times2[i])


23 9.999999999999998e-08 8.724152220000001e-08
25 1.4260633500000002e-07 1.1744051100000001e-07
26 1.7616076700000004e-07 1.476394998e-07
27 2.43269631e-07 1.778384886e-07
28 3.0072491952222505e-07 2.382364662e-07
29 3.581802080444501e-07 2.9259464604e-07
30 4.156354965666751e-07 3.4695282588e-07
31 4.7309078508890013e-07 4.0131100572e-07
32 5.880013621333502e-07 4.5566918556e-07
33 6.914208814733553e-07 5.6438554524e-07
34 8.100091496488332e-07 6.62230268952e-07
35 9.167385910067633e-07 7.711583242590333e-07
36 9.999999999999997e-07 8.691935740353633e-07
37 1.0234680323646935e-06 9.672288238116933e-07
38 1.1301974737226238e-06 9.999999999999997e-07
39 1.343656356438484e-06 1.0652640735880232e-06
40 1.5571152391543442e-06 1.163299323364353e-06
41 1.7705741218702045e-06 1.359369822917013e-06
42 1.984033004586065e-06 1.555440322469673e-06
43 2.4109507700177854e-06 1.817291000051389e-06
44 2.837868535449506e-06 2.079141677633105e-06
45 3.2647863008812263e-06 2.340992355214821e-06
46 3.691

In [118]:
# update the line widths
# t = times1[-1]
t = None
t1 = 92
t2 = 92


# Update the nodes
slope = -max_node_pen_width / np.log10(concentration_tol)
for index in nodes1:
#     species = species_list1[index]
    species = universal_species_list[index]
    if re.search(r'^[a-zA-Z0-9_]*$', str(species)) is not None:
        species_string = str(species)
    else:
        # species name contains special characters                
        species_string = '"{0}"'.format(str(species))

    node = graph.get_node(species_string)[0]
    concentration = universal_concentrations1[t1, index] / max_concentration1
    if concentration < concentration_tol:
        penwidth = 0.0
    else:
        penwidth = round(slope * np.log10(concentration) + max_node_pen_width, 3)
    node.set_penwidth(penwidth)


for index in nodes2:
#     species = species_list2[index]
    species = universal_species_list[index]
    if re.search(r'^[a-zA-Z0-9_]*$', str(species)) is not None:
        species_string = str(species)
    else:
        # species name contains special characters                
        species_string = '"{0}"'.format(str(species))

    node = graph.get_node(species_string)[0]
    concentration = universal_concentrations2[t2, index] / max_concentration2
    if concentration < concentration_tol:
        penwidth = 0.0
    else:
        penwidth = round(slope * np.log10(concentration) + max_node_pen_width, 3)
    node.set_penwidth(penwidth)
    
# Update the edges
slope = -max_edge_pen_width / np.log10(species_rate_tol)
for index in range(len(edges1)):
    reactant_index, product_index = edges1[index]
    if reactant_index in nodes1 and product_index in nodes1:
#         reactant = species_list1[reactant_index]
#         product = species_list1[product_index]
        reactant = universal_species_list[reactant_index]
        product = universal_species_list[product_index]

        if re.search(r'^[a-zA-Z0-9_]*$', str(reactant)) is not None:
            reactant_string = str(reactant)
        else:
            reactant_string = '"{0}"'.format(str(reactant))

        if re.search(r'^[a-zA-Z0-9_]*$', str(product)) is not None:
            product_string = str(product)
        else:
            product_string = '"{0}"'.format(str(product))

        edge = graph.get_edge(reactant_string, product_string)[0]
        # Determine direction of arrow based on sign of rate
        species_rate = species_rates1[t1, reactant_index, product_index] / max_species_rate1
        if species_rate < 0:
            edge.set_dir("back")
            species_rate = -species_rate
        else:
            edge.set_dir("forward")
        # Set the edge pen width
        if species_rate < species_rate_tol:
            penwidth = 0.0
            edge.set_dir("none")
        else:
            penwidth = round(slope * np.log10(species_rate) + max_edge_pen_width, 3)
        edge.set_penwidth(penwidth)

for index in range(len(edges2)):
    reactant_index, product_index = edges2[index]
    if reactant_index in nodes2 and product_index in nodes2:
#         reactant = species_list2[reactant_index]
#         product = species_list2[product_index]
        reactant = universal_species_list[reactant_index]
        product = universal_species_list[product_index]

        if re.search(r'^[a-zA-Z0-9_]*$', str(reactant)) is not None:
            reactant_string = str(reactant)
        else:
            reactant_string = '"{0}"'.format(str(reactant))

        if re.search(r'^[a-zA-Z0-9_]*$', str(product)) is not None:
            product_string = str(product)
        else:
            product_string = '"{0}"'.format(str(product))

        edge = graph.get_edge(reactant_string, product_string)[0]
        # Determine direction of arrow based on sign of rate
        species_rate = species_rates2[t2, reactant_index, product_index] / max_species_rate2
        if species_rate < 0:
            edge.set_dir("back")
            species_rate = -species_rate
        else:
            edge.set_dir("forward")
        # Set the edge pen width
        if species_rate < species_rate_tol:
            penwidth = 0.0
            edge.set_dir("none")
        else:
            penwidth = round(slope * np.log10(species_rate) + max_edge_pen_width, 3)
        edge.set_penwidth(penwidth)

In [119]:
graph.write_dot('mechanism_flux.dot')
graph.write_png('mechanism_flux.png')
img = mpimg.imread('mechanism_flux.png')
imgplot = plt.imshow(img)

In [120]:
# delete all edges
for nodeA in graph.get_nodes():
    for nodeB in graph.get_nodes():
        graph.del_edge(nodeA, nodeB)
        graph.del_edge(nodeA, nodeB)

# remake plot 1:
for reactant_index, product_index in edges1:
    if reactant_index in nodes and product_index in nodes:
        reactant = universal_species_list[reactant_index]
        product = universal_species_list[product_index]
        edge = pydot.Edge(str(reactant), str(product), color='#ff000080')
        edge.set_penwidth(max_edge_pen_width)
        graph.add_edge(edge)
        
        
graph.write_dot('mechanism_flux1.dot')
graph.write_png('mechanism_flux1.png')
img = mpimg.imread('mechanism_flux1.png')
imgplot = plt.imshow(img)

In [121]:
# delete all edges
for nodeA in graph.get_nodes():
    for nodeB in graph.get_nodes():
        graph.del_edge(nodeA, nodeB)
        graph.del_edge(nodeA, nodeB)
        
# remake plot 2:
for reactant_index, product_index in edges2:
    if reactant_index in nodes and product_index in nodes:
        reactant = universal_species_list[reactant_index]
        product = universal_species_list[product_index]
        edge = pydot.Edge(str(reactant), str(product), color='#0000ff80')
        edge.set_penwidth(max_edge_pen_width)
        graph.add_edge(edge)
        
graph.write_dot('mechanism_flux2.dot')
graph.write_png('mechanism_flux2.png')
img = mpimg.imread('mechanism_flux2.png')
imgplot = plt.imshow(img)

In [ ]:
dir(graph)

In [ ]:
nodes